In [229]:
## Commands used to fix torchtext.legacy library issue - restart runtime after installing these
# !pip install torch --quiet
# !pip uninstall torchtext -y
# !pip install torchtext==0.9.0 --quiet

import torch
from torchtext.legacy.data import Field, LabelField, BucketIterator

SEED = 1234

torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

TEXT = Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')
LABEL = LabelField(dtype = torch.float)


In [230]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [231]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [232]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [233]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [234]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [235]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [236]:
print(dir(TEXT))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_first', 'build_vocab', 'dtype', 'dtypes', 'eos_token', 'fix_length', 'ignore', 'include_lengths', 'init_token', 'is_target', 'lower', 'numericalize', 'pad', 'pad_first', 'pad_token', 'postprocessing', 'preprocess', 'preprocessing', 'process', 'sequential', 'stop_words', 'tokenize', 'tokenizer_args', 'truncate_first', 'unk_token', 'use_vocab', 'vocab', 'vocab_cls']


In [237]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202832), (',', 192261), ('.', 165265), ('and', 109487), ('a', 109331), ('of', 100653), ('to', 94063), ('is', 76154), ('in', 61259), ('I', 54021), ('it', 53587), ('that', 49391), ('"', 44355), ("'s", 43356), ('this', 42293), ('-', 37602), ('/><br', 35737), ('was', 34974), ('as', 30437), ('with', 30063)]


In [238]:
from torchtext.legacy import data

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,  # can have different batch sizes for train, validate and test
    device = device)

In [239]:
print('Created `torchtext_train_dataloader` with %d batches!'%len(train_iterator))  # BATCH_SIZE * 274 (batches from this print) = 17500 (total train samples)

Created `torchtext_train_dataloader` with 274 batches!


In [240]:
import sys
counter = 1
for batch_no, batch in enumerate(train_iterator):
    print(batch_no, '\n', batch, '\n')
    # print(text, batch_len)
    print(batch.label.shape,'\n', batch.text.shape)
    if counter == 1:
      break

0 
 
[torchtext.legacy.data.batch.Batch of size 64]
	[.text]:[torch.LongTensor of size 868x64]
	[.label]:[torch.FloatTensor of size 64] 

torch.Size([64]) 
 torch.Size([868, 64])


In [241]:
print(len(TEXT.vocab))  # we have total 25002 words i.e vocab size in Train

25002


In [242]:
import torch.nn as nn

import sys
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        print('1sid', '\n', text.shape, len(text[0]))

        embedded = self.embedding(text)
        print('2sid', embedded.shape)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        print('3sid', output.shape, hidden.shape)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [243]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
# print(model)

In [244]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data.shape)  # 5*3 not 3*5; same is issue with hidden weights


The model has 2,592,105 trainable parameters
embedding.weight torch.Size([25002, 100])
rnn.weight_ih_l0 torch.Size([256, 100])
rnn.weight_hh_l0 torch.Size([256, 256])
rnn.bias_ih_l0 torch.Size([256])
rnn.bias_hh_l0 torch.Size([256])
fc.weight torch.Size([1, 256])
fc.bias torch.Size([1])


In [245]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [246]:
criterion = nn.BCEWithLogitsLoss()  # this includes a sigmoid activatio in the last layer along with a BCE loss

In [247]:
model = model.to(device)
criterion = criterion.to(device)

In [248]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [249]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [250]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [251]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [252]:
N_EPOCHS = 5

best_valid_loss = float('inf')
import sys
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    sys.exit

1sid 
 torch.Size([1827, 64]) 64
2sid torch.Size([1827, 64, 100]) tensor([[[ 2.3605, -1.2176,  0.3657,  ...,  0.1005,  0.1764, -0.0845],
         [ 2.6920,  1.3048,  0.6084,  ...,  0.9604,  0.6976,  1.0929],
         [-0.9031,  1.1818,  0.7339,  ..., -0.4830, -0.0064,  0.5000],
         ...,
         [-0.9057, -0.8280,  0.1663,  ..., -1.3351, -0.1762, -0.3888],
         [-0.2222,  0.8854,  0.1469,  ..., -0.9625,  0.0565, -0.4714],
         [ 1.9146,  0.9768,  1.0811,  ...,  0.8111, -0.2699, -2.0615]],

        [[ 0.6344, -0.7282, -0.9697,  ..., -0.0389,  0.7072,  0.7395],
         [-1.1012,  0.4145,  2.0096,  ..., -0.1087, -0.7862, -1.1002],
         [-0.2222,  0.8854,  0.1469,  ..., -0.9625,  0.0565, -0.4714],
         ...,
         [-0.5290, -0.2811,  1.6160,  ..., -0.2792,  0.8981,  1.5703],
         [ 1.1086,  0.0241, -0.9880,  ..., -0.3208,  0.8984,  0.2799],
         [-0.1664,  0.2926,  1.3934,  ...,  0.0664, -3.1663,  0.2765]],

        [[-0.0618, -1.8870,  0.7666,  ...,  0.9934

KeyboardInterrupt: ignored